In [3]:
def genetic_algorithm(data, thresholds, classifiers, regressors, population_size, generations, crossover_prob, mutation_prob):
    '''

    Funnction to select the chromosome representing the most effective set of weights for the trading strategy thresholds, as evaluated by the backtesting process
    Returns: list containing the weights for the strategies

    Parameters
    ----------
    weights : list
        A list containing the weights for the strategies
    thresholds : list 
        List containing thresholds 
    classifiers: list 
        List containing the classifiers 
    regressors: list 
        List containing the regressors

    population_size : integer
        Number of initial options of best solutions
    generations : integer
        How often should we run the algorithm
    crossover_prob : float (range 0 - 1) 
        Probability to combine genes of the parents, rather high 
    mutation_prob = float (range 0-1)
        Probability to randomly mutate, rather low

    -----------
    '''
    
    optimizer = Optimize_eps(data, thresholds: list, classifiers: list, regressors: list)
    num_thresholds = len(thresholds)
    num_chromosomes = len(thresholds)

    # 1. First generation population is initialized
    population = initialize_population(num_thresholds, num_chromosomes, population_size)

    # iterate over desired generations
    for generation in range(generations):

        # 2. Evaluate the fitness of each chromosome (=[sharpe, ret, vol])
        fitnesses = [optimizer.evaluate(chromosome) for chromosome in population]
        normalized_fitnesses = np.array([f[0] for f in fitnesses]) / sum(f[0] for f in fitnesses)

        # 3. from new population with random parent selection and two children that are passed to the next generation 
        new_population = []
        for _ in range(population_size // 2):
            # Select two chromosomes
            parents = np.random.choice(len(population), size=2, p=normalized_fitnesses)
            parent1, parent2 = population[parents[0]], population[parents[1]]

            # 4. CROSSOVER: generate random number between 0 and 1 to decide if this chromosome will be recombined
            if random.random() < crossover_prob:
                # choose random crossover point between the first gene (index 1) and the last gene 
                point = random.randint(1, num_chromosomes - 1)
                child1 = parent1[:point] + parent2[point:]
                child2 = parent2[:point] + parent1[point:]
            else:
                child1, child2 = parent1, parent2
            
            # 5. MUTATION: Iterate over each gene and mutate if random number is lower than mutation probability
            child1 = [gene if random.random() > mutation_prob else random.uniform(0.0, 1.0) for gene in child1]
            child2 = [gene if random.random() > mutation_prob else random.uniform(0.0, 1.0) for gene in child2]
            
            # New population is formed
            new_population.extend([child1, child2])

        population = new_population
        best_fitness = max(f[0] for f in fitnesses)
        print(f"Generation {generation}: Best Fitness = {best_fitness}")

    best_chromosome = population[np.argmax([f[0] for f in fitnesses])]
    
    return best_chromosome 

